## Permutation Test using One-Way ANOVA in CUDA

In [4]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin:/usr/local/cuda/bin


### Random Dataset Generation

In [2]:
!python -m pip install scikit-learn
!python -m pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (12.4 MB)


In [5]:
from sklearn.datasets import make_classification

k = 3
N = 6
X, y = make_classification(
    n_samples = N,             # row number
    n_features = 5,            # feature numbers
    n_informative = 3,         # The number of informative features
    n_redundant = 0,           # The number of redundant features
    n_repeated = 0,            # The number of duplicated features
    n_classes = k,             # The number of classes 
    n_clusters_per_class = 1,  # The number of clusters per class
    random_state = 42,         # random seed 
    scale=100                  # scale of the data
)

In [6]:
import pandas as pd

df = pd.concat([pd.DataFrame(X)[[0]], pd.DataFrame(y).astype(int)], axis=1)
df.columns = [0, 1]
df = df.sort_values(1).reset_index().iloc[:,1:]
df.to_csv("dataset.csv", header=False, index=False)

#### C Version (Serial)

In [7]:
%%writefile c_permutation_anova.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define MAX_LINE 1024

/* Linear Congruential Generator (LCG) */
unsigned int lcg_random(unsigned int seed) {
    return (1103515245U * (seed) + 12345U) & 0x7fffffffU;
}

/* Fisher–Yates Shuffling Algorithm */
void permute(size_t *array, size_t N, unsigned int seed, size_t *result) {
    for (size_t i = 0; i < N; i++) {
        result[i] = array[i];
    }
    for (size_t i = N - 1; i > 0; i--) {
        size_t j = lcg_random(seed) % (i + 1);  // pick random index [0, i]
        size_t temp = result[i];
        result[i] = result[j];
        result[j] = temp;
    }
}
/* One Way ANOVA */
double OneWayAnova(size_t N, int k, size_t *n_i, size_t *group, double *feature){
    /* AVERAGE & GROUP AVERAGE */
    double *group_ave = (double *) calloc(k, sizeof(double));
    double average = 0.0;
    for (int i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    for (int i = 0; i < k; i++) {
        group_ave[i] /= n_i[i];
    }

    /* SUM OF SQUARED ERROR (SSE) */
    double SSE = 0.0;
    double temp;
    for (int i = 0; i < N; i++) {
        temp = feature[i] - group_ave[group[i]];
        SSE += temp*temp;
    }

    /* SSR (SUM OF SQUARED RESIDUALS) */
    double SSR = 0.0;
    for (int i = 0; i < k; i++) {
        temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }
    free(group_ave);
    /* F-statistic */
    return (SSR/(k-1))/(SSE/(N-k));
}
int main() {
    size_t perm_count;
    size_t N;   // number of rows
    size_t k;   // number of groups
    clock_t start, end;
    size_t counter = 10;

    /* GET THE NUMBER OF ROWS */
    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);
    printf("Number of Permutations: ");
    scanf("%zu", &perm_count);

    double *feature = (double*) malloc(N * sizeof(double));
    size_t *group = (size_t*) malloc(N * sizeof(size_t));
    size_t *temp_group = (size_t*) malloc(N * sizeof(size_t));
    size_t *n_i = (size_t*) calloc(k, sizeof(size_t));
    double *F_dist = (double*) malloc(perm_count * sizeof(double));

    /* READ THE DATA */
    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL){
        perror("Error opening file");
        return 1;
    }

    char line[MAX_LINE];
    size_t i = 0;
    while (fgets(line, sizeof(line), fp)) {
        if (i >= N) break;  // prevent overflow

        line[strcspn(line, "\n")] = 0;

        char *token = strtok(line, ",");
        int j = 0;
        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token); // convert to float and save
            else {
                group[i] = atoi(token); // convert to int and save
                if (group[i] >= k){
                    perror("Error group count");
                    return 1;
                }
                n_i[group[i]] += 1;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);
     // fill-in cache
    permute(group, N, i, temp_group);
    OneWayAnova(N, k, n_i, group, feature);

    // Execution time start here: CPU Permutation
    /* CPU PERMUTATION */
    double elapse, time_taken;
    elapse = 0.0f;
    
    for (int c=0; c<counter; c++){
        start = clock();
        for (size_t i = 0; i < perm_count; i++) {
            if (i == 0)
                F_dist[i] = OneWayAnova(N, k, n_i, group, feature);
            else 
                F_dist[i] = OneWayAnova(N, k, n_i, temp_group, feature);
            
            // Always permute from the ORIGINAL group array
            permute(group, N, i, temp_group);
        }
        end = clock();
        time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
    }
    printf("\nFunction (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", counter, elapse/counter, perm_count);



    // Print first 5 permutations
    printf("First 5 permutations\n");
     for (size_t i = 0; i < 5; i++) {
        printf("CPU Permutation %zu: ", i+1);
        for (size_t j = 0; j < N; j++) {
            if (i == 0)
                printf("%zu ", group[j]);
            else
                printf("%zu ", temp_group[j]);
        }
        printf("\n");
    }

    printf("\nSTEP 3: Printing First 5 and Last 5 Results\n");
    for (int i = 0; i < 5; i++) {
        printf ("F_dist %d: %lf\n", i, F_dist[i]);
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        printf ("F_dist %d: %lf\n", i, F_dist[i]);
    } 

    size_t extreme_count = 0;
    double p_value = 0.0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
       }
    }
    printf ("Extreme Count: %lu\n", extreme_count);
    p_value = (double)extreme_count / perm_count;
    printf("p-value: %lf\n", p_value);

    /* free memory */
    free(feature);
    free(group);
    free(n_i);
    free(F_dist);

    return 0;
}

Writing c_permutation_anova.c


In [8]:
%%bash
gcc c_permutation_anova.c -o c_permutation_anova -lm

In [9]:
%%bash
./c_permutation_anova < input.txt

Number of Rows: Number of Groups: Number of Permutations: 
Function (in C) average time for 10 loops is 207.039600 milliseconds to execute an array size 1000000 
First 5 permutations
CPU Permutation 1: 0 0 1 1 2 2 
CPU Permutation 2: 2 1 2 0 0 1 
CPU Permutation 3: 2 1 2 0 0 1 
CPU Permutation 4: 2 1 2 0 0 1 
CPU Permutation 5: 2 1 2 0 0 1 

STEP 3: Printing First 5 and Last 5 Results
F_dist 0: 0.005957
F_dist 1: 2.508775
F_dist 2: 1.975504
F_dist 3: 2.508775
F_dist 4: 1.398650
F_dist 999995: 0.071876
F_dist 999996: 1.975504
F_dist 999997: 2.508775
F_dist 999998: 0.071876
F_dist 999999: 0.020986
Extreme Count: 983330
p-value: 0.983330


#### CUDA Version (Parallel)

In [10]:
%%writefile cuda_permutation_anova_1.cu
// WORKING VERSION
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define MAX_LINE 1024

/* Linear Congruential Generator (LCG) */
__device__ unsigned int lcg_random(unsigned int seed) {
    return (1103515245U * (seed) + 12345U) & 0x7fffffffU;
}

/* Fisher–Yates Shuffling Algorithm */
__device__ void permute(size_t *array, size_t N, unsigned int seed, size_t *result) {
    for (size_t i = 0; i < N; i++) {
        result[i] = array[i];
    }
    for (size_t i = N - 1; i > 0; i--) {
        size_t j = lcg_random(seed) % (i + 1);
        size_t temp = result[i];
        result[i] = result[j];
        result[j] = temp;
    }
}

/* One Way ANOVA */
__device__ double OneWayAnova(size_t N, int k, size_t *n_i, size_t *group, double *feature){
    
    double group_ave[100];
    for (int i = 0; i < k; i++) {
        group_ave[i] = 0.0;
    }
        
    double average = 0.0;
    for (int i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    for (int i = 0; i < k; i++) {
        group_ave[i] /= n_i[i];
    }

    /* SUM OF SQUARED ERROR (SSE) */
    double SSE = 0.0;
    double temp;
    for (int i = 0; i < N; i++) {
        temp = feature[i] - group_ave[group[i]];
        SSE += temp*temp;
    }

    /* SSR (SUM OF SQUARED RESIDUALS) */
    double SSR = 0.0;
    for (int i = 0; i < k; i++) {
        temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }

    /* F-statistic */
    return (SSR/(k-1))/(SSE/(N-k));
}

__global__ void gpu_permute(size_t *array, size_t perm_count, size_t N, size_t *perm_array) {
    int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = thread_id; i < perm_count; i += stride) {
        if (i != 0) {
            permute(array, N, i-1, &perm_array[i * N]);
        } else {
            for (int j = 0; j < N; j++) {
                perm_array[j] = array[j];
            }
        }
    }
}

__global__ void gpu_anova(size_t *perm_array, size_t N, int k, size_t perm_count, double *feature, size_t *n_i, double *F_dist) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int perm_idx = idx; perm_idx < perm_count; perm_idx += stride) {
        size_t *current_group = &perm_array[perm_idx * N];
        F_dist[perm_idx] = OneWayAnova(N, k, n_i, current_group, feature);
    }
}

int main() {
    size_t perm_count;
    size_t N;
    size_t k;
    size_t counter = 10;

    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);
    printf("Number of Permutations: ");
    scanf("%zu", &perm_count);

    // Get GPU device
    int device = -1;
    cudaGetDevice(&device);

    // Memory allocation
    double *feature;
    size_t *group;
    size_t *n_i;
    size_t *perm_array;
    double *F_dist;

    cudaMallocManaged(&feature, N * sizeof(double));
    cudaMallocManaged(&group, N * sizeof(size_t));
    cudaMallocManaged(&n_i, k * sizeof(size_t));
    cudaMallocManaged(&perm_array, N * perm_count * sizeof(size_t));
    cudaMallocManaged(&F_dist, perm_count * sizeof(double));

    // Initialize n_i to zero
    memset(n_i, 0, k * sizeof(size_t));

    // MEMORY ADVISE: Set up for input data (feature, group, n_i)
    cudaMemAdvise(feature, N * sizeof(double), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(group, N * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(n_i, k * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);


    // Prefetch data to CPU memory
    cudaMemPrefetchAsync(feature, N * sizeof(double), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), cudaCpuDeviceId, NULL);


    // READ DATA FROM FILE
    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL){
        perror("Error opening file");
        return 1;
    }

    char line[MAX_LINE];
    size_t i = 0;
    while (fgets(line, sizeof(line), fp)) {
        if (i >= N) break;

        line[strcspn(line, "\n")] = 0;

        char *token = strtok(line, ",");
        int j = 0;
        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token);
            else {
                group[i] = atoi(token);
                if (group[i] >= k){
                    perror("Error group count");
                    fclose(fp);
                    return 1;
                }
                n_i[group[i]] += 1;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);
    
    // PREFETCH: Move input data to GPU before computation
    cudaMemPrefetchAsync(feature, N * sizeof(double), device, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), device, NULL);
    
    // Prefetch output arrays to GPU
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), device, NULL);

    // Wait for prefetch to complete
    cudaDeviceSynchronize();

    // Number of Threads and Blocks
    size_t numThreads = 256;
    size_t numBlocks = (perm_count + numThreads - 1) / numThreads;

    // STEP 1: GPU PERMUTATION
    printf("\nSTEP 1: Generating Permutations\n");
    printf("Launching kernel with %zu blocks and %zu threads per block\n", numBlocks, numThreads);
    
    for (size_t c = 0; c < counter; c++){
        gpu_permute<<<numBlocks, numThreads>>>(group, perm_count, N, perm_array);
    }
    cudaDeviceSynchronize();
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), device, NULL);
    cudaDeviceSynchronize();

    // STEP 2: GPU ANOVA
    printf("\nSTEP 2: Computing F-statistic\n");
    printf("Launching kernel with %zu blocks and %zu threads per block\n", numBlocks, numThreads);
    
    for (size_t c = 0; c < counter; c++){
        gpu_anova<<<numBlocks, numThreads>>>(perm_array, N, k, perm_count, feature, n_i, F_dist);
    }
    cudaDeviceSynchronize();

    // PREFETCH: Move results back to CPU for printing
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), cudaCpuDeviceId, NULL);

    // PRINT RESULTS
    printf("\nDEBUG: First 5 permutations\n");
    for (int i = 0; i < 5; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N && j < 20; j++) {  // Limit output for readability
            printf("%zu ", perm_array[i * N + j]);
        }
        if (N > 20) printf("...");
        printf("\n");
    }

    printf("\n Printing Results\n");
    printf("First 5 F-statistics:\n");
    for (int i = 0; i < 5; i++) {
        printf("F_dist[%d]: %lf\n", i, F_dist[i]);
    }

    // Calculate p-value
    size_t extreme_count = 0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
        }
    }
    double p_value = (double)extreme_count / (double)perm_count;
    printf("\nOriginal F-statistic: %lf\n", F_dist[0]);
    printf("Extreme count: %zu out of %zu permutations\n", extreme_count, perm_count);
    printf("p-value: %lf\n", p_value);
    

    // Free memory
    cudaFree(feature);
    cudaFree(group);
    cudaFree(n_i);
    cudaFree(perm_array);
    cudaFree(F_dist);

    return 0;
}

Writing cuda_permutation_anova_1.cu


In [11]:
%%bash
nvcc cuda_permutation_anova_1.cu -o cuda_permutation_anova_1 -Wno-deprecated-gpu-targets

In [12]:
%%bash
nvprof ./cuda_permutation_anova_1 < input.txt

==1157746== NVPROF is profiling process 1157746, command: ./cuda_permutation_anova_1


Number of Rows: Number of Groups: Number of Permutations: 
STEP 1: Generating Permutations
Launching kernel with 3907 blocks and 256 threads per block

STEP 2: Computing F-statistic
Launching kernel with 3907 blocks and 256 threads per block

DEBUG: First 5 permutations
GPU Permutation 1: 0 0 1 1 2 2 
GPU Permutation 2: 1 2 2 0 0 1 
GPU Permutation 3: 0 1 2 1 2 0 
GPU Permutation 4: 0 2 2 1 1 0 
GPU Permutation 5: 2 1 0 0 1 2 

 Printing Results
First 5 F-statistics:
F_dist[0]: 0.005957
F_dist[1]: 2.508775
F_dist[2]: 1.975504
F_dist[3]: 2.508775
F_dist[4]: 1.398650

Original F-statistic: 0.005957
Extreme count: 983330 out of 1000000 permutations
p-value: 0.983330


==1157746== Profiling application: ./cuda_permutation_anova_1
==1157746== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   72.37%  5.0697ms        10  506.97us  504.06us  511.67us  gpu_permute(unsigned long*, unsigned long, unsigned long, unsigned long*)
                   27.63%  1.9351ms        10  193.51us  192.49us  197.29us  gpu_anova(unsigned long*, unsigned long, int, unsigned long, double*, unsigned long*, double*)
      API calls:   94.98%  1.02266s         5  204.53ms  13.495us  1.02200s  cudaMallocManaged
                    3.71%  39.938ms        12  3.3281ms  21.797us  31.435ms  cudaMemPrefetchAsync
                    0.64%  6.8500ms         4  1.7125ms  6.4380us  4.9426ms  cudaDeviceSynchronize
                    0.34%  3.6326ms         5  726.53us  30.494us  2.1487ms  cudaFree
                    0.21%  2.2401ms        20  112.01us  6.0970us  2.0031ms  cudaLaunchKernel
                    0.04%  413.